# Create a Grafana Data Source and Dashboard

Follow the steps in this this tutorial to create a "stocks" Grafana dashboard using Iguazio data source <br>
Note that you need to create a Grafana service before running the notebook <br>
Creating a grafana service can be done from the "services" screen in the Platform

In [1]:
import nuclio

## Environment

In [2]:
%nuclio cmd -c python -m pip install git+https://github.com/v3io/grafwiz.git

## Function

In [3]:
# nuclio: start-code

In [4]:
from grafwiz import *
import os

In [5]:
def deploy_dashboard(context, 
                     grafana_url:str = 'http://grafana', 
                     streamview_url:str = 'http://3.12.231.36:30100',
                     v3io_container:str = 'bigdata',
                     stocks_kv_table:str = 'stocks/stocks_kv',
                     stocks_tsdb_table:str = 'stocks/stocks_tsdb',
                     stocks_sentiment_tsdb_table:str = 'stocks/stocks_sentiment_tsdb'):
    # Create a new Grafana dashboard
    DataSource('stream-viewer', frames_url=streamview_url).deploy(grafana_url)
    
    dash = Dashboard("stocks-test", start='now-1d', dataSource='Iguazio')

    # Add a symbol combo box (template) with data from the stocks table
    dash.template(name="SYMBOL", label="Symbol", query="fields=symbol;table=stocks/stocks_kv;backend=kv;container=bigdata")

    # Create a table and log viewer in one row
    tbl = Table('tbl1',span=8).source(table=stocks_kv_table,fields=['symbol','volume', 'price','last_updated'],container=v3io_container)
    log = Table('log', dataSource='stream-viewer', span=4)
    dash.row([tbl,log])

    # Create 2 charts on the second row
    dash.row([Graph(metric).series(table=stocks_tsdb_table, fields=[metric], filter='symbol=="$SYMBOL"',container=v3io_container) for metric in ['price','volume']])
    dash.row([Graph('sentiment').series(table=stocks_sentiment_tsdb_table, fields=['sentiment'], filter='symbol=="$SYMBOL"', container=v3io_container)])
    
    # Deploy to Grafana
    return dash.deploy(grafana_url)

In [6]:
# nuclio: end-code

## Local test

In [7]:
from mlrun import code_to_function, run_local

fn = code_to_function('grafana_dashboard_deployer',
                      kind='job',
                      handler='deploy_dashboard',
                      project='stocks')
fn.spec.build.base_image = 'mlrun/mlrun'
fn.export('06-grafana.yaml')

> 2020-09-24 07:56:06,957 [info] function spec saved to path: 06-grafana.yaml


In [8]:
run_local(handler=deploy_dashboard)

> 2020-09-24 07:56:06,969 [warning] artifact path is not defined or is local, artifacts will not be visible in the UI
> 2020-09-24 07:56:06,977 [info] starting run mlrun-7fb3ca-deploy_dashboard uid=f3939d1a0b84494784036c8ff2f8e62e  -> http://mlrun-api:8080
Datasource stream-viewer already exists
Datasource stream-viewer created successfully
Dashboard stocks-test created successfully


project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
default,...f2f8e62e,0,Sep 24 07:56:07,completed,mlrun-7fb3ca-deploy_dashboard,v3io_user=adminkind=handlerowner=adminhost=jupyter-dcfbc696d-s229k,,,,


to track results use .show() or .logs() or in CLI: 
!mlrun get run f3939d1a0b84494784036c8ff2f8e62e --project default , !mlrun logs f3939d1a0b84494784036c8ff2f8e62e --project default
> 2020-09-24 07:56:07,142 [info] run executed, status=completed
